In [8]:
!pip install folium geopy rtree matplotlib mapclassify geojson
!pip install geopandas


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import folium
import geopandas as gpd
import pandas as pd
import os
import geojson
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster, Fullscreen, MousePosition

/home/malobato/.local/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
def load_shapefile(name, cols=None, index=None, rename=None):
    path = os.path.abspath(f"./data/{name}.zip")
    df = gpd.read_file(f'zip://{path}')

    if cols:
        df = df[cols]
    if rename:
        df = df.rename(columns=rename)
    if index:
        df = df.set_index(index)
    return df.to_crs(epsg=4326)
    

def add_barrios(m):
    for name, r in barrios.iterrows():
        # Without simplifying the representation of each borough,
        # the map might not be displayed
        sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda x: {'fillColor': 'orange'})
        folium.Popup(f"<b>Barrio:</b> <span style='white-space: nowrap;'>{name}</span>").add_to(geo_j)
        geo_j.add_to(m)
    m



def get_map(withbarrios=True):
    #tiles maps: https://github.com/python-visualization/folium/tree/main/folium/templates/tiles
    # Create a base map
    m = folium.Map(location=[40.422, -3.696], tiles='stamentoner', zoom_start=12)
    m.add_child(Fullscreen())
    m.add_child(MousePosition())
    
    if withbarrios:
        add_barrios(m)
    
    return m

In [4]:
# Barrios and distritos Dataset
lambda_rename = lambda x:x.upper().replace("Á", "A").replace("É", "E").replace("Í", "I").replace("Ó", "O").replace("Ú", "U")
poblacion_barrios = pd.read_csv("./data/poblacion-barrios.csv").set_index("NOMBRE").to_dict()["Total"]

barrios = load_shapefile("barrios", ["NOMBRE", "geometry"])
distritos = load_shapefile("distritos", ["DISTRI_MAY", "geometry"], rename={"DISTRI_MAY": "NOMBRE"}, index="NOMBRE")

barrios["NOMBRE"] = barrios.apply(lambda x:lambda_rename(x['NOMBRE']), axis=1)
barrios["poblacion"] = barrios.apply(lambda x:int(poblacion_barrios[x['NOMBRE']]), axis=1)

barrios = barrios.set_index("NOMBRE")

In [5]:
# Load all Datasets
bus_interurbano_estaciones = load_shapefile("bus-interurbano-estaciones", 
                                            cols=["IDPOSTE", "geometry"], 
                                            rename={"IDPOSTE": "NOMBRE"},
                                            index="NOMBRE")
bus_interurbano_estaciones["TYPE"] = "bus-interurbano-estaciones"
bus_interurbano_estaciones["latitude"] = bus_interurbano_estaciones.apply(lambda x: x['geometry'].y, axis=1)
bus_interurbano_estaciones["longitude"] = bus_interurbano_estaciones.apply(lambda x: x['geometry'].x, axis=1)

bus_urbano_madrid_estaciones = load_shapefile("bus-urbano-madrid-estaciones", 
                                              cols=["IDPOSTE", "geometry"], 
                                              rename={"IDPOSTE": "NOMBRE"},
                                              index="NOMBRE")
bus_urbano_madrid_estaciones["TYPE"] = "bus-urbano-madrid-estaciones"
bus_urbano_madrid_estaciones["latitude"] = bus_urbano_madrid_estaciones.apply(lambda x: x['geometry'].y, axis=1)
bus_urbano_madrid_estaciones["longitude"] = bus_urbano_madrid_estaciones.apply(lambda x: x['geometry'].x, axis=1)

bus_urbano_no_madrid_estaciones = load_shapefile("bus-urbano-no-madrid-estaciones", 
                                                 cols=["IDPOSTE", "geometry"], 
                                                 rename={"IDPOSTE": "NOMBRE"},
                                                 index="NOMBRE")
bus_urbano_no_madrid_estaciones["TYPE"] = "bus-urbano-no-madrid-estaciones"
bus_urbano_no_madrid_estaciones["latitude"] = bus_urbano_no_madrid_estaciones.apply(lambda x: x['geometry'].y, axis=1)
bus_urbano_no_madrid_estaciones["longitude"] = bus_urbano_no_madrid_estaciones.apply(lambda x: x['geometry'].x, axis=1)
bus_urbano_estaciones = pd.concat([bus_urbano_madrid_estaciones, bus_urbano_no_madrid_estaciones])

cercanias_estaciones = load_shapefile("cercanias-estaciones", 
                                      cols=["DENOMINACI", "geometry"], 
                                      rename={"DENOMINACI": "NOMBRE"}, 
                                      index="NOMBRE")
cercanias_estaciones["TYPE"] = "cercanias-estaciones"
cercanias_estaciones["latitude"] = cercanias_estaciones.apply(lambda x: x['geometry'].y, axis=1)
cercanias_estaciones["longitude"] = cercanias_estaciones.apply(lambda x: x['geometry'].x, axis=1)

metro_estaciones = load_shapefile("metro-estaciones", 
                                  cols=["DENOMINACI", "geometry"], 
                                  rename={"DENOMINACI": "NOMBRE"}, 
                                  index="NOMBRE")
metro_estaciones["TYPE"] = "metro-estaciones"
metro_estaciones["latitude"] = metro_estaciones.apply(lambda x: x['geometry'].y, axis=1)
metro_estaciones["longitude"] = metro_estaciones.apply(lambda x: x['geometry'].x, axis=1)

metro_ligero_estaciones = load_shapefile("metro-ligero-estaciones", 
                                         cols=["DENOMINACI", "geometry"], 
                                         rename={"DENOMINACI": "NOMBRE"}, 
                                         index="NOMBRE")
metro_ligero_estaciones["TYPE"] = "metro-ligero-estaciones"
metro_ligero_estaciones["latitude"] = metro_ligero_estaciones.apply(lambda x: x['geometry'].y, axis=1)
metro_ligero_estaciones["longitude"] = metro_ligero_estaciones.apply(lambda x: x['geometry'].x, axis=1)

#bus_interurbano_estaciones, bus_urbano_estaciones
#cercanias_estaciones, metro_estaciones, metro_ligero_estaciones
#pd.concat([A, B])



In [6]:
#bus_interurbano_estaciones, bus_urbano_estaciones
#cercanias_estaciones, metro_estaciones, metro_ligero_estaciones
#pd.concat([A, B])

df = bus_urbano_estaciones

# Show bicimad stations as heatmap
m = get_map()
HeatMap(data=df[['latitude', 'longitude']], radius=20).add_to(m)
m

In [49]:
#bus_interurbano_estaciones, bus_urbano_estaciones
#cercanias_estaciones, metro_estaciones, metro_ligero_estaciones
#pd.concat([bus_interurbano_estaciones, bus_urbano_estaciones, cercanias_estaciones, metro_estaciones, metro_ligero_estaciones])

df = pd.concat([bus_interurbano_estaciones, bus_urbano_estaciones, cercanias_estaciones, metro_estaciones, metro_ligero_estaciones])

#calculate dictionaries
estaciones_barrios = gpd.sjoin(barrios, df).reset_index()
estaciones_barrios = estaciones_barrios[["NOMBRE"]].groupby(['NOMBRE'])['NOMBRE'].count().squeeze().to_dict()
estaciones_data = {key: {
    "poblacion": poblacion_barrios[key],
    "estaciones": val,
    "estaciones/1000": float(val)*1000.0/float(poblacion_barrios[key])
} for key,val in estaciones_barrios.items()}


estaciones_cp = {key: val["estaciones/1000"] for key,val in estaciones_data.items()}

# Create a base map
m = get_map(withbarrios=False)


# Add a choropleth map to the base map
cp = Choropleth(geo_data=barrios.__geo_interface__,
           data=estaciones_cp, 
           key_on="feature.id", 
           fill_color="OrRd", 
           legend_name='Bicimad Station docks per district',
           nan_fill_color = "LightGrey"
          ).add_to(m)

# add popup
for s in cp.geojson.data['features']:
    s['properties']['NOMBRE'] = s['id']
    s['properties']['estaciones'] = estaciones_data.get(s['id'], {}).get("estaciones", 0)
    s['properties']['estaciones/1000'] = round(estaciones_data.get(s['id'], {}).get("estaciones/1000", 0), 2)
    
folium.GeoJsonTooltip(['NOMBRE', 'poblacion', 'estaciones', 'estaciones/1000']).add_to(cp.geojson)
folium.LayerControl().add_to(m)

# Display the map
cp.geojson

In [54]:
import branca.colormap as cm
import statistics
vmax,vmedian = round(max(estaciones_cp.values()) * 100), round(statistics.median(estaciones_cp.values()) * 100)
colormap = cm.LinearColormap(colors=['orange','red','darkred'], index=[0,vmedian,vmax], vmin=0, vmax=vmax).to_step(n=10)

#plotting
m = get_map(withbarrios=False)

folium.GeoJson(
    cp.geojson.data,
    style_function=lambda feature: {
        'fillColor': 'red' if feature['properties']['estaciones/1000'] == 0 else colormap(feature['properties']['estaciones/1000']*100),
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)
colormap.add_to(m)



m